In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd; pd.set_option('precision', 10)
import numpy as np; np.set_printoptions(precision=10, suppress=True)
from scipy import stats

import scipy.optimize as opt

import datetime as dt

import matplotlib.pyplot as plt
import pickle

In [5]:
start_train = '1999-12-31'
end_train = '2015-01-01'
start_test = '2015-01-02'
end_test = '2020-05-19'

import os 
cwd = os.getcwd()
print(os.getcwd())
os.chdir('/home/math/PycharmProjects/hmm-master')

/home/math/PycharmProjects/hmm-master/notebooks


In [6]:
df = pd.read_csv('data/price_series.csv', index_col = 'Time')

df.index = pd.to_datetime(df.index)

In [10]:
df.dropna(inplace=True)

,Hedge Funds Global,MSCI World,MSCI Emerging Markets,DAX,European Public Real Estate,S&P 500,Oil,Gold index,Barclays US Treasury,Private Equity,T-bills rf
Time,,,,,,,,,,,
2003-11-21,1081.38,955.98,403.72,4340.10510,1429.80084,1035.28,611.6554,395.50,1326.99,100.00,418.1791
2003-11-24,1081.32,965.54,401.04,4397.43380,1431.74973,1052.08,575.4708,391.90,1321.79,100.31,418.2065
2003-11-25,1083.40,968.17,405.50,4403.26222,1433.16327,1053.89,576.0513,391.75,1324.57,101.89,418.2212
2003-11-26,1085.95,974.72,408.90,4426.98605,1438.55764,1058.45,588.4353,396.00,1320.80,103.45,418.2358
2003-11-28,1087.50,976.02,413.33,4491.39405,1452.12089,1058.20,588.4353,398.35,1315.73,103.36,418.2777
...,...,...,...,...,...,...,...,...,...,...,...
2021-02-08,1403.01,2795.89,1399.81,16937.97358,2530.72534,3915.59,85.8871,1835.25,2520.90,178.10,527.6745
2021-02-09,1405.78,2798.55,1409.14,16961.28390,2550.13614,3911.23,86.4832,1839.60,2521.58,178.59,527.6700
2021-02-10,1407.11,2800.68,1422.94,16911.83899,2553.47106,3909.88,86.9580,1842.65,2526.54,179.92,527.6643
